In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
from os.path import join, isfile
from copy import deepcopy

In [ ]:
# load one local library with additional statistics for the xarray datastructure
import xhydrostats as xs

## plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(10,6))

sel = dict(time=slice('01-01-1995', '09-01-1996'))
sel = dict(time=slice('01-01-1991', '09-01-1991'))
da_wl.isel(index=0, ensemble=1).sel(**sel).plot.line(x='time', ax=ax)
xr.where(np.logical_and(peaks100, peaks.sum('scen')==2), peaks_val, np.nan).isel(index=0, ensemble=1).sel(**sel).to_series().plot(ax=ax, linewidth=0, markersize=10, marker='x')

fig2, ax2 = plt.subplots(1,1,figsize=(10,6))
# ds_cmf['river_discharge'].isel(index=0, ensemble=1).sel(**sel).plot.line(x='time', ax=ax2)
ds_cmf['sea_surge'].isel(index=0).sel(**sel).plot.line(x='time', ax=ax2)
# ds_cmf['sea_water_level_climatology'].isel(index=0).sel(**sel).plot.line(x='time', ax=ax2)

In [ ]:
%matplotlib inline
driver = 'compound'
cfi_driver = cmpnd_stats[f'peaks_{driver}_perc']*100
cfi_driver.to_series().unstack(-1).T.plot.box(whis=[0,95], showfliers=False)

In [ ]:
driver = 'compound'
cfi_driver = cmpnd_stats[f'cfi_{driver}']
cfi_driver.to_series().unstack(-1).T.plot.box(whis=[0,95], showfliers=False)

In [ ]:
# test for single station
%matplotlib inline
import matplotlib.pyplot as plt
q = 95
min_dist = 30
rid = 387
# rid = 3200


ds_i= ds_cmf.isel(index=rid).sel(ensemble='nerc').squeeze().load()

# components
da_cmpnd = ds_i['water_level'].sel(scen='cmpnd') 
da_surge = ds_i['water_level'].sel(scen='surge')
da_runoff = ds_i['water_level'].sel(scen='runoff')
da_surge_wdwmax = da_surge.rolling(time=7, min_periods=1, center=True).max()
da_runoff_wdwmax = da_runoff.rolling(time=7, min_periods=1, center=True).max()
da_cmpnd_std = da_cmpnd.std('time')
cfi_surge = ((da_cmpnd - da_runoff_wdwmax) / da_cmpnd_std)
cfi_runoff = ((da_cmpnd - da_surge_wdwmax) / da_cmpnd_std)
cfi_runoff

threshold = xs.nanpercentile(da_cmpnd, q=q, dim='time')
ts = xs.peaks_over_threshold(da_cmpnd, min_dist=min_dist, threshold=threshold, dim='time') 
cfi_surge_peaks = cfi_surge.where(xr.ufuncs.isfinite(ts))
cfi_runoff_peaks = cfi_runoff.where(xr.ufuncs.isfinite(ts))

fig, ((ax1, ax2, ax3), (ax12, ax22, ax32)) = plt.subplots(2, 3, figsize=(15, 9), sharex=True)
sel = dict(time=slice('07-01-2002', '07-01-2003'))
ymax = max(ds_i['water_level'].sel(**sel).max().values, ds_i['sea_water_level'].sel(**sel).max().values)
ymin = min(ds_i['water_level'].sel(**sel).min().values, ds_i['sea_water_level'].sel(**sel).min().values)
ylim = [ymin, ymax]
ylab = 'water surface elevation [m+EGM96]'

# da_cmpnd.sel(**sel).plot(ax=ax2, c='red')
ds_i['water_level'].sel(**sel, scen='surge').plot(ax=ax2, x='time', label='surge', c='blue')
ds_i['water_level'].sel(**sel, scen='runoff').plot(ax=ax2, x='time', label='runoff', c='green')
ds_i['water_level'].sel(**sel, scen='cmpnd').plot(ax=ax2, x='time', label='compound', c='red')
cfi_surge.sel(**sel).plot(ax=ax22, c='blue', linestyle='--')
cfi_runoff.sel(**sel).plot(ax=ax22, c='green', linestyle='--')
ax2.set_title('river mouth')
ax22.set_title('')
ax2.set_ylabel(ylab)
ax2.set_ylim(ylim)
ax2.legend()


ds_i['river_discharge'].sel(**sel).plot(ax=ax3, c='green')
ds_i['river_discharge_climatology'].sel(**sel).plot(ax=ax3, c='green', linestyle='--')

ax3.set_title('upstream boundary')
ax3.set_ylabel('discharge [m3/s]')

ds_i['sea_water_level_climatology'].sel(**sel).plot(ax=ax1, c='blue', linestyle='--')
ds_i['sea_water_level'].sel(**sel).plot(ax=ax1, c='blue')
ax1.set_title('downstream boundary')
ax1.set_ylabel(ylab)
ax1.set_ylim(ylim)

# ds_i['sea_water_level_climatology'].sel(**sel).plot(ax=ax1, c='magenta', x='time')

# if ts.sel(**sel).dropna('time').values.size>1: # needs two dates to plot
#     ts.sel(**sel).dropna('time').plot(c='r', marker='*', markersize=9, linewidth=0, ax=ax)
#     cfi_runoff_peaks.sel(**sel).dropna('time').plot(c='r', marker='*', markersize=9, linewidth=0, ax=ax2)


# cfi_peaks.mean('time').compute().values, ds_i.to_series().unstack(0).tail(5)